### 1. Chances of getting 7 when throwing two dices

#### The winning combinations:

- 1-6
- 2-5
- 3-4
- 4-3
- 5-2 
- 6-1

#### Total number of combinations for two dices
d1 = 6 ways for dice 1

d2 = 6 ways for dice 2

d1 x d2 = 36

P(Of a seven) = 6/36 = 1/6

In [1]:
# %%bash
!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import random
from tabulate import tabulate

In [8]:
DICE_VALUES = list(range(1,7))

def roll_a_dice():
    return random.choice(DICE_VALUES)


def draw_a_number_in_two_rolls(number):
    dice_1 = roll_a_dice()
    dice_2 = roll_a_dice()
    return (dice_1 + dice_2) == number
         

def monte_carlo_roll_number(num_of_experiments, sum_dice):
    wins = 0.0
    for _ in range(num_of_experiments):
        win = draw_a_number_in_two_rolls(sum_dice)
        if win:
            wins += 1
    prob_winning = (wins / num_of_experiments) * 100
    return prob_winning
 

In [9]:
print('Estimated probability of a rolling a seven for several repetitions.\n')
exp_sizes = [10_000, 100_000, 1_000_000, 10_000_000]
target_value = 7
print('  {0:.2f}% Calculated'.format(100.0/6))

for exp_size in exp_sizes:
    prob = monte_carlo_roll_number(exp_size, target_value)
    exp_size_str = '{0:,}'.format(exp_size)
    print('  {0:.2f}% (Experiment size = {1:})'.format(prob, exp_size_str))


Estimated probability of a rolling a seven for several repetitions.

  16.67% Calculated
  17.17% (Experiment size = 10,000)
  16.93% (Experiment size = 100,000)
  16.66% (Experiment size = 1,000,000)
  16.68% (Experiment size = 10,000,000)


### 2. Chance of winning for each sum.

In [10]:
def calculate_estimates_for_dice_sums(target_values):
    exp_size = 1_000_000
    estimations = dict()
    for target_value in target_values:
        prob = monte_carlo_roll_number(exp_size, target_value)
        estimations[target_value] = prob
    return estimations

target_values = list(range(2,13)) 
print('Possible values for sum of two dices = {0}'.format(','.join([str(x) for x in target_values])))
# Save the estimations for later use.
monte_carlo_estimations = calculate_estimates_for_dice_sums(target_values)
print(' ')
print('Estimations for each sum of two dices:', end='')

 
estimate_table = []
for target_value in sorted(monte_carlo_estimations.keys()):
    prob = monte_carlo_estimations[target_value]
    estimate_table.append((target_value, '{0:5.2f}%'.format(prob)))

print(' ')
print(tabulate(estimate_table, headers=["Sum of dices", "Estimate"], tablefmt="mixed_outline"))
                            

Possible values for sum of two dices = 2,3,4,5,6,7,8,9,10,11,12
 
Estimations for each sum of two dices: 
  Sum of dices  Estimate
--------------  ----------
             2  2.76%
             3  5.56%
             4  8.32%
             5  11.12%
             6  13.85%
             7  16.62%
             8  13.91%
             9  11.13%
            10  8.36%
            11  5.53%
            12  2.76%


#### 3. Possible dice combinations for each sum value

In [11]:
from collections import defaultdict

print('Possible combinations for each sum:')
print(' ')
results = defaultdict(list)
for d1 in range(1,7):
    for d2 in range(1,7):
       results[d1+d2].append((d1,d2)) 
result_table = []
for key in sorted(results.keys()):
    dice_combinations = ', '.join([str(x) for x in results[key]])
    result_table.append((key, dice_combinations))
    
print(' ')
print(tabulate(result_table, headers=["Sum of dices", "Dice combinations"], tablefmt="mixed_outline"))
                            

Possible combinations for each sum:
 
 
  Sum of dices  Dice combinations
--------------  ----------------------------------------------
             2  (1, 1)
             3  (1, 2), (2, 1)
             4  (1, 3), (2, 2), (3, 1)
             5  (1, 4), (2, 3), (3, 2), (4, 1)
             6  (1, 5), (2, 4), (3, 3), (4, 2), (5, 1)
             7  (1, 6), (2, 5), (3, 4), (4, 3), (5, 2), (6, 1)
             8  (2, 6), (3, 5), (4, 4), (5, 3), (6, 2)
             9  (3, 6), (4, 5), (5, 4), (6, 3)
            10  (4, 6), (5, 5), (6, 4)
            11  (5, 6), (6, 5)
            12  (6, 6)


#### 4. Comparing 'calculated' vs 'estimated' probabilities

In [12]:
print("Comparing 'calculated' vs 'estimated' probabilities.")
print(' ')
basic_prob = 100.0/36 
comparison_table = []
for key in sorted(results.keys()):
    calculated_value = len(results[key]) * basic_prob
    estimated_value = monte_carlo_estimations[key]
    difference_value = calculated_value - estimated_value
    comparison_table.append(
        (f'{calculated_value:.3f}%', 
         f'{estimated_value:.3f}%',
         f'{difference_value:.3f}%')
    )
print(tabulate(
    comparison_table, headers=["Calculated", "Estimated", "Delta"], 
    colalign=("right","right", "right"),
    tablefmt="mixed_outline"))


Comparing 'calculated' vs 'estimated' probabilities.
 
  Calculated    Estimated    Delta
------------  -----------  -------
      2.778%       2.760%   0.018%
      5.556%       5.560%  -0.005%
      8.333%       8.318%   0.016%
     11.111%      11.124%  -0.013%
     13.889%      13.851%   0.038%
     16.667%      16.616%   0.050%
     13.889%      13.905%  -0.016%
     11.111%      11.130%  -0.019%
      8.333%       8.358%  -0.025%
      5.556%       5.531%   0.025%
      2.778%       2.760%   0.018%
